# Taipei.Py August : MoviePy
by Aaron Lai

## 安裝

- pip install moviepy
- 安裝 ImageMagick 處理文字： http://www.imagemagick.org/script/binary-releases.php
 - MacOS : brew install imagemagick
 - Linux : sudo apt-get install imagemagick

## Some Words About MoviePy
### Based on numpy 計算 影像 與 聲音
### 1.  影像：以 "Clip" 物件為基礎延伸出：VideoFileClip, ImageClip, TextClip, ColorClip 等等 
### 2.  聲音：AudioFileClip, AudioArrayClip( numpy_array, fps=44100 )
### 剪接影片、上字幕、 做GIF 等

<img src="demo.jpg">

### Import MoviePy：

In [ ]:
from moviepy.editor import *

### First Step : 載入影片
- 載入影片： VideoFileClip( 影片路徑 )

字型設定： 中文 - "WeibeiTCB"
#### 可以自行指定字型檔路徑  font = 'path_to_tff'

In [ ]:
# 載入影片
Truth = VideoFileClip('truth.mp4')

# 我們字型用 WeibeiTCB 才有中文
font = "WeibeiTCB"

### Screen 1: 文字 TextClip

- 字體大小 fontsize
- 顏色 color  --->  接受 Hex color ex. #ffffff 或是常見顏色 ex. red, cyan, yellow
- 外邊顏色 stroke_color
- 外邊寬度 stroke_width
- 自型 font
- 設定位置 .set_position(座標/文字) (或是.set_pos())
- 設定顯示長度 .set_duration(秒)
- 設定開始時間 .set_start(秒)

#### 影片控制：

- 擷取中間 .subclip(start, end) 可用秒數或是 "01:04:32"
- 疊合短片 CompositeVideoClip([clip1, clip2...])
- 淡入效果 .fadein(秒數)

In [ ]:
texts = [" 你     的時候腦袋有曾冒出過我嗎？", "  自慰                          "]

# TextClip 處理文字
sub_text1 = TextClip(texts[0], fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_position((120, 600))

sub_text2 = TextClip(texts[1], fontsize=75, color='red', font=font).set_position((120, 592))

# 合成兩個文字
text_1 = CompositeVideoClip([sub_text1, sub_text2], size=Truth.size).set_duration(3)

# 可以連續疊 methods (ex. clip.set_pos().set_duration()...)
text_2 = TextClip(" 自慰的時候啊...", fontsize=60, color='cyan', stroke_color="#000000",
                  stroke_width=1, font=font).set_pos(("center", 600)).set_duration(1).set_start(3)

# 組合所有 clips
screen_1 = CompositeVideoClip([Truth.subclip(0, 4.5), text_1, text_2]).fadein(1)

ipython_display(screen_1, width=480, fps=15)

### Screen 2: 影片物理特效與圖片

影片特效作法：
- new_clip = clip.fx( vfx.effect_func, parameter1=... ) 


- new_clip = effect_func( clip, parameter1=... )

MoviePy 內建特效： http://zulko.github.io/moviepy/ref/videofx.html

也可以自訂特效函數！(對每個影格的 RGB array 做計算)

Screen2 特效:

- 鏡像反射 vfx.mirror_x, vfx.mirror_y
- 組合畫面 clips_array([ [ clip_0, clip_1 ], [ clip_2, clip_3 ] ])
- 縮放大小 clip.resize(...) 可以指定倍數 ex. 0.5, 可以設定長寬 (1280, 720), 或是等比例縮放 width=1280 
- 切畫面 vfx.crop( x1 = 100, x2 = 500, y1 = 200, y2 = 400 )
- 調整亮度 vfx.colorx, factor = 1.5 變亮 1.5 倍 ( < 1 則是變暗 )
- 調整速度 vfx.speed, factor = 0.4 變為 0.4 倍速

#### 影片控制

- 圖片 ImageClip(檔案路徑)
- 串接影片 concatenate_videoclips

In [ ]:
text_3 = TextClip(" 有！", fontsize=80, color='red', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 590))

# 正常
clip0 = CompositeVideoClip([Truth.subclip(4.5, 6), text_3.set_duration(1.5)])

# 把螢幕切分四個子畫面 每個子畫面都做鏡像反射 ( mirror_x, mirror_y ) 再組合
clip = CompositeVideoClip([Truth.subclip(4.8, 5.7), text_3.set_duration(0.9)])
clip1 = clips_array([[clip.fx(vfx.mirror_x), clip],
                     [clip.fx(vfx.mirror_y).fx(vfx.mirror_x), clip.fx(vfx.mirror_y)]]).resize(clip.size)

# 切畫面 再resize回原大小 最後把亮度增加 1.5 倍 
clip2 = clip.fx(vfx.crop, x1=60, y1=30, x2=860, y2=480).resize(clip.size).fx(vfx.colorx, 1.5)

# 將影片速度放慢到 0.4 倍 再和圖片合成
img = ImageClip('like.png').resize(2)
clip3 = CompositeVideoClip([clip.fx(vfx.speedx, 0.4), img.set_pos((700, 50)).set_duration(1.5).set_start(0.5)])

# 將上面四個短片 串接 成一個影片
screen_2 = concatenate_videoclips([clip0, clip1, clip2, clip3])

ipython_display(screen_2, width=480, fps=15)

### Screen 3

- 調整聲音大小 .volumex( 倍數 )
- resize進階用法：隨時間而變 .resize( lambda t: f ( t ) )

In [ ]:
# ##################### 字幕 #####################
text_4 = TextClip(" 有喔", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1).set_start(2)

text_5 = TextClip(" 正點！", fontsize=80, color='yellow', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1).set_start(3.2)
# ##################### 字幕 #####################

# 圖片逐漸放大
img = ImageClip('power.jpg').set_duration(1.5).set_start(0.5).resize(lambda t: 1 + 0.3*t).set_pos((50, 100))

# 音量調為兩倍
screen_3 = CompositeVideoClip([Truth.subclip(6, 10.2), text_4, text_5, img]).volumex(2)

ipython_display(screen_3, width=480, fps=15)

### Screen 4


In [ ]:
# ##################### 字幕 #####################
text_6 = TextClip(" 《真心話喝一杯》前男女友篇", fontsize=70, color='yellow', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(2)

text_7 = TextClip(" 八月份Taipei.Py", fontsize=80, color='red', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 10)).set_duration(2)

text_8 = TextClip(" 前任情侶隨機問彼此一系列問題", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(2.85).set_start(2)

text_9 = TextClip(" 他們可以選擇回答問題或是喝一杯", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(2.5).set_start(4.85)

text_10 = TextClip(" 我們曾經是男女朋友", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1).set_start(10)

text_11 = TextClip(" 我們是短暫的夏日愛火", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(2).set_start(11)

text_12 = TextClip(" 我們交往一年半", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1.5).set_start(13)

text_13 = TextClip(" 她和我的室友約會過", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1.5).set_start(15)

text_14 = TextClip(" 我和他的死黨約過一次會", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(2).set_start(16.5)

text_15 = TextClip(" 才會認識他", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1.5).set_start(18.5)

text_16 = TextClip(" 然後九個月之後 我們開始交往", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(2.5).set_start(20)
# ##################### 字幕 #####################

screen_4 = CompositeVideoClip([Truth.subclip(10.2, "00:00:32.9"), text_6, text_7,
                               text_8, text_9, text_10, text_11, text_12, text_13,
                               text_14, text_15, text_16])

ipython_display(screen_4, width=480, fps=15)

### Screen 5

In [ ]:
# ##################### 字幕 #####################
text_17 = TextClip(" 你敢不敢把酒放在我身上然後喝光", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(3).set_start(0.5)

text_18 = TextClip(" 如果你不敢 你就要罰兩杯", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(3).set_start(3.5)

text_19 = TextClip(" 什麼意思啊(內心暗爽)", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1).set_start(6.5)

text_20 = TextClip(" 放身上是哪招", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1.2).set_start(7.5)

text_21 = TextClip(" 例如放在你的肚臍上喝嗎", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(2.8).set_start(8.7)

text_22 = TextClip(" 不", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(0.8).set_start(13.5)

text_23 = TextClip(" 她灑出來了！", fontsize=70, color='red', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1).set_start(14.3)

text_24 = TextClip(" 我喝不了啦 (ＸＤ)", fontsize=70, color='yellow', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1).set_start(19.5)

text_25 = TextClip(" 好 來吧 我來了！", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1.5).set_start(21)

text_26 = TextClip(" 頭往右邊轉", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1).set_start(23.5)

text_27 = TextClip(" 不是不是 那是妳的左邊", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1).set_start(25)
# ##################### 字幕 #####################

img2 = ImageClip('shy.png').resize(0.5).set_pos((780, 260)).set_duration(1).set_start(29.7)

screen_5 = CompositeVideoClip([Truth.subclip("00:02:15", "00:02:47"), text_17, text_18,
                               text_19, text_20, text_21, text_22, text_23, text_24,
                               text_25, text_26, text_27, img2])

ipython_display(screen_5, width=480, fps=15)

### Screen 6

In [ ]:
# ##################### 字幕 #####################
text_28 = TextClip(" 喔 拜託不要...", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1.2)

text_29 = TextClip(" 你自慰的時候腦袋有曾冒出過我嗎？", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(2.3).set_start(1.2)

text_30 = TextClip(" 拜託喝了吧", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1.2).set_start(3.5)

text_31 = TextClip(" 爸拖喝吧 OTZ", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1.2).set_start(5.7)

text_32 = TextClip(" 不管答案是什麼我都不想知道", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1.8).set_start(7.2)

text_33 = TextClip(" 謝謝你", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1).set_start(10.3)

text_34 = TextClip(" 有啊！        ", fontsize=70, color='yellow', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 595)).set_duration(0.6).set_start(12.4)

text_35 = TextClip("        真的嗎？", fontsize=90, color='red', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 580)).set_duration(1.3).set_start(12.7)

text_36 = TextClip("            有嗎？", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1).set_start(14)

text_37 = TextClip(" 有時候會啊              ", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1).set_start(14.5)

text_38 = TextClip(" 妳知道妳可以打給我的 (A__A)", fontsize=70, color='yellow', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 595)).set_duration(1.5).set_start(16.2)

text_39 = TextClip(" 當妳想到的時候", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1).set_start(18)

text_40 = TextClip(" 才不要！", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 595)).set_duration(1).set_start(19.5)

text_41 = TextClip(" 這是... 這是我『獨處』的時間！", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 595)).set_duration(2).set_start(20.5)
# ##################### 字幕 #####################

img3 = ImageClip('shy.png').resize(0.7).set_pos((720, 120)).set_duration(1.8).set_start(21)

screen_6 = CompositeVideoClip([Truth.subclip("00:03:41.7", "00:04:04.7"), text_28, text_29,
                               text_30, text_31, text_32, text_33, text_34, text_35,
                               text_36, text_37, text_38, text_39, text_40, text_41, img3])

ipython_display(screen_6, width=480, fps=15)

### Screen 7 - 1

In [ ]:
# ##################### 字幕 #####################
text_42 = TextClip(" 妳敢不敢...", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1.8).set_start(0.4)

text_43 = TextClip(" 跟我接吻", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1.5).set_start(3.3)

text_44 = TextClip(" 什麼？！", fontsize=70, color='yellow', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 595)).set_duration(0.8).set_start(9)
# ##################### 字幕 #####################

screen_7_1 = CompositeVideoClip([Truth.subclip("00:04:55.7", "00:05:05.7"), text_42, text_43, text_44])

ipython_display(screen_7_1, width=480, fps=15)

### Screen 7 - 2
- 時間鏡像(倒著播) vfx.time_mirror 
- 淡出 fadeout( 秒數 )

In [ ]:
# ##################### 字幕 #####################
text_45 = TextClip(" 我寧願親你的嘴也不要喝那杯純的威士忌", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 580)).set_duration(3.1).set_start(0.2)

text_46 = TextClip(" 好 那就來吧！", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(0.8).set_start(3.6)

text_47 = TextClip(" 好 來吧", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(0.8).set_start(4.4)

text_48 = TextClip(" 來啊", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(0.6).set_start(5.6)

text_49 = TextClip(" 1 2 3 來！", fontsize=80, color='red', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 590)).set_duration(1).set_start(6.2)

text_50 = TextClip(" 喝酒囉！", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1).set_start(17)

text_51 = TextClip(" 我們應該給你們拍拍手", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 510)).set_duration(2.3).set_start(18)

text_52 = TextClip(" 好朋友啦", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1).set_start(22)

text_53 = TextClip(" 敬友誼！", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1.5).set_start(23.5)

text_54 = TextClip(" 還有親親", fontsize=60, color='cyan', stroke_color="#000000", stroke_width=1,
                 font=font).set_pos(("center", 600)).set_duration(1.5).set_start(25.5)
# ##################### 字幕 #####################

screen_7_2 = CompositeVideoClip([Truth.subclip("00:05:24.3", "00:05:53"), text_45, text_46, text_47,
                                 text_48, text_49, text_50, text_51, text_52, text_53, text_54])

sub_clip = Truth.subclip("00:05:50.3", "00:05:53")
sub_clip2 = sub_clip.fx(vfx.time_mirror)

# 加速 與 時間鏡像 與 變亮 與 淡出
clip1 = concatenate_videoclips([sub_clip2.speedx(1.2), sub_clip.speedx(1.5).fx(vfx.colorx, 1.2)])
clip2 = concatenate_videoclips([sub_clip2.speedx(1.8).fx(vfx.colorx, 1.3), sub_clip.speedx(2).fx(vfx.colorx, 1.5)])
ending = concatenate_videoclips([clip1, clip2]).fadeout(1)

screen_7_2 = concatenate_videoclips([screen_7_2, ending])

ipython_display(screen_7_2, width=480, fps=15)

### Final Step : 串接影片 大功告成
- 輸出 .write_video( myvideo.mp4, fps=24 )

可輸出 .mkv, .mp4, .....

可供設定： fps, codec, bitrate, audio=True, audio_fps, audio_nbytes, audio_codec, audio_bitrate.....
#### 輸出時間正比於解析度！ 輸出 720p 約為 360p 的 4 倍時間

- ipython_display 有影片長度限制 120 秒，設定 maxduration 以擺脫限制

In [ ]:
final_movie = concatenate_videoclips([screen_1, screen_2, screen_3, screen_4, screen_5, screen_6,
                                      screen_7_1, screen_7_2])

# 設定 maxduration
ipython_display(final_movie, width=480, fps=15, maxduration=1000)

In [ ]:
final_movie.write_videofile("taipeipy_demo.mp4", fps=24)

### 更多詳見官網： http://zulko.github.io/moviepy/index.html